In [1]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.1 MB/s eta 0:00:00


In [2]:
!pip install pyautogen

In [3]:
!pip install dask[dataframe]

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.2/243.2 kB 3.9 MB/s eta 0:00:00


In [8]:
import autogen
from autogen import AssistantAgent, UserProxyAgent # group chat, supervisor agent
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
config_list = [{"model": "gpt-4o", "api_key":OPENAI_API_KEY}]

gpt4o_config = {
    "cache_seed": 42, # change the cache_seed for different trials
    "temperature": 0,
    "config_list": config_list,
    "timeout": 120
}

llm_config = {"config_list": config_list}


In [5]:
task = '''
 **Task**: As an architect, you are required to design a solution for the
 following business requirements:
    - Data storage for massive amounts of IoT data
    - Real-time data analytics and machine learning pipeline
    - Scalability
    - Cost Optimization
    - Region pairs in Europe, for disaster recovery
    - Tools for monitoring and observability
    - Timeline: 6 months

    Break down the problem using a Chain-of-Thought approach. Ensure that your
    solution architecture is following best practices.
    '''

cloud_prompt = '''
**Role**: You are an expert cloud architect. You need to develop architecture proposals
using either cloud-specific PaaS services, or cloud-agnostic ones.
The final proposal should consider all 3 main cloud providers: Azure, AWS and GCP, and provide
a data architecture for each. At the end, briefly state the advantages of cloud over on-premises
architectures, and summarize your solutions for each cloud provider using a table for clarity.
'''
cloud_prompt += task

oss_prompt = '''
**Role**: You are an expert on-premises, open-source software architect. You need
to develop architecture proposals without considering cloud solutions.
 Only use open-source frameworks that are popular and have lots of active contributors.
 At the end, briefly state the advantages of open-source adoption, and summarize your
 solutions using a table for clarity.
'''
oss_prompt += task

lead_prompt =  '''
**Role**: You are a lead Architect tasked with managing a conversation between
the cloud and the open-source Architects.
Each Architect will perform a task and respond with their resuls. You will critically
review those and also ask for, or point to, the disadvantages of their solutions.
You will review each result, and choose the best solution in accordance with the business
requirements and architecture best practices. You will use any number of summary tables to
communicate your decision.
'''
lead_prompt += task

In [6]:
# creating the main supervisor agent
user_proxy = UserProxyAgent(
    name="supervisor",
    system_message="A Human Head of Architecture",
    code_execution_config={
        "last_n_messages": 2,
        "work_dir": "groupchat",
        "use_docker": False
    },
    human_input_mode="NEVER"
)

# cloud architect agent
cloud_agent = AssistantAgent(
    name="cloud",
    system_message=cloud_prompt,
    llm_config={"config_list": config_list}
)

# main developer agent
oss_agent = AssistantAgent(
    name="oss",
    system_message=oss_prompt,
    llm_config={"config_list": config_list}
)

# lead architect agent
lead_agent = AssistantAgent(
    name="lead",
    system_message=lead_prompt,
    llm_config={"config_list": config_list}
)

In [7]:
# flow of delivery ->
# supervisor_agent -> cloud_agent -> oss_agent -> lead_agent
def state_transition(last_speaker, groupchat):
  messages = groupchat.messages

  if last_speaker is user_proxy:
    return cloud_agent
  elif last_speaker is cloud_agent:
    return oss_agent
  elif last_speaker is oss_agent:
    return lead_agent
  elif last_speaker is lead_agent:
    return None

In [10]:
# initialize the chat

group_chat = autogen.GroupChat(
    agents = [user_proxy, cloud_agent, oss_agent, lead_agent],
    messages = [],
    max_round = 6,
    speaker_selection_method = state_transition
)

manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_config)

In [11]:
user_proxy.initiate_chat(
    manager, message="Provide your best architecture based on the AI agent for the bussiness requirements." # you can customize this bussiness requirement as per your need
)

supervisor (to chat_manager):

Provide your best architecture based on the AI agent for the bussiness requirements.

--------------------------------------------------------------------------------

Next speaker: cloud

cloud (to chat_manager):

To design a robust architecture that meets all the business requirements, we need to address each requirement systematically while considering cloud-specific and cloud-agnostic elements. Here is how we can approach it for each cloud provider:

### 1. Data Storage for Massive Amounts of IoT Data

For storing IoT data efficiently, we need a solution that is durable, scalable, and supports high throughput.

- **Azure**: Use Azure Blob Storage for cost-effective, scalable storage. Consider Azure Data Lake Storage for advanced analytics needs.
- **AWS**: Use Amazon S3 with intelligent tiering to manage large volumes of data affordably. S3 is highly durable and scales automatically.
- **GCP**: Use Google Cloud Storage, which offers similar scalabilit

ChatResult(chat_id=None, chat_history=[{'content': 'Provide your best architecture based on the AI agent for the bussiness requirements.', 'role': 'assistant', 'name': 'supervisor'}, {'content': 'To design a robust architecture that meets all the business requirements, we need to address each requirement systematically while considering cloud-specific and cloud-agnostic elements. Here is how we can approach it for each cloud provider:\n\n### 1. Data Storage for Massive Amounts of IoT Data\n\nFor storing IoT data efficiently, we need a solution that is durable, scalable, and supports high throughput.\n\n- **Azure**: Use Azure Blob Storage for cost-effective, scalable storage. Consider Azure Data Lake Storage for advanced analytics needs.\n- **AWS**: Use Amazon S3 with intelligent tiering to manage large volumes of data affordably. S3 is highly durable and scales automatically.\n- **GCP**: Use Google Cloud Storage, which offers similar scalability and durability features. It also integra